In [1]:
from __future__ import annotations

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

try:
    from tqdm import tqdm
except Exception:
    def tqdm(x, **kwargs):
        return x

import yfinance as yf


# ============================================================
# Exit Strategy (STATIC INPUTS) !!!
# ============================================================
CONFIG = {
    # Universe
    "TICKERS": ['TMUS', 'SO', 'WMT', 'T', 'PGR', 'NEE', 'PM', 'BSX', 'AVGO', 'FTNT'],
    "MARKET_TICKER": "^GSPC",

    # Download Period
    "DOWNLOAD_START": "2022-01-01",
    "DOWNLOAD_END": None,

    # Single Live Check Date (YYYY-MM-DD)
    "AS_OF_DATE": "2024-11-01",

    # Windows (Month)
    "BETA_ALPHA_WINDOW": 12,
    "RISK_WINDOW": 12,

    # Monthly Risk Free Rate (0 if unknown)
    "RF_MONTHLY": 0.0,

    # Thresholds (tuning)
    "ALPHA_3M_DROP_THRESHOLD": -0.002,
    "ALPHA_RANK_DROP_POINTS": 0.20,
    "SHARPE_DROP_THRESHOLD": -0.30,
    "IR_DROP_THRESHOLD": -0.30,
    "VOL_INCREASE_THRESHOLD": 0.20,

    # Momentum
    "MOM_LOOKBACK": 6,
    "MA_WINDOW": 6,

    # Action
    "ACTION_REDUCE": True,

    # Extra output: How many months of prices to show
    "SHOW_LAST_N_MONTHS_PRICES": 6,
}


# ============================================================
# Helpers
# ============================================================
def _safe_float(x) -> float:
    try:
        if isinstance(x, (pd.Series, pd.DataFrame)) and hasattr(x, "iloc"):
            return float(x.iloc[0])
        return float(x)
    except Exception:
        return np.nan


def _get_company_name(ticker: str) -> str:
    try:
        info = yf.Ticker(ticker).info
        return info.get("shortName") or info.get("longName") or info.get("displayName") or ""
    except Exception:
        return ""


def _to_monthly_adjclose(price: pd.DataFrame) -> pd.DataFrame:
    # month-end last available business day
    return price.resample("M").last()


def _monthly_returns(monthly_px: pd.DataFrame) -> pd.DataFrame:
    return monthly_px.pct_change()


def _rolling_beta_alpha(stock_ret: pd.Series, mkt_ret: pd.Series, window: int = 12):
    """
    Rolling CAPM (simple): r_i = a + b * r_m + e
    returns: beta, alpha (intercept) as series
    """
    idx = stock_ret.index.intersection(mkt_ret.index)
    y = stock_ret.loc[idx].astype(float)
    x = mkt_ret.loc[idx].astype(float)

    beta = pd.Series(index=idx, dtype=float)
    alpha = pd.Series(index=idx, dtype=float)

    for i in range(window - 1, len(idx)):
        sl = idx[i - window + 1 : i + 1]
        yy = y.loc[sl].dropna()
        xx = x.loc[sl].dropna()
        common = yy.index.intersection(xx.index)
        yy = yy.loc[common]
        xx = xx.loc[common]

        if len(common) < max(6, window // 2):
            beta.iloc[i] = np.nan
            alpha.iloc[i] = np.nan
            continue

        vx = np.var(xx.values, ddof=1)
        if vx <= 0:
            beta.iloc[i] = np.nan
            alpha.iloc[i] = np.nan
            continue

        b = np.cov(xx.values, yy.values, ddof=1)[0, 1] / vx
        a = yy.mean() - b * xx.mean()
        beta.iloc[i] = b
        alpha.iloc[i] = a

    return beta, alpha


def _rolling_sharpe(ret: pd.Series, rf_monthly: float = 0.0, window: int = 12) -> pd.Series:
    ex = ret - rf_monthly
    mu = ex.rolling(window).mean()
    sd = ex.rolling(window).std(ddof=1)
    return mu / sd


def _rolling_information_ratio(stock_ret: pd.Series, mkt_ret: pd.Series, window: int = 12) -> pd.Series:
    idx = stock_ret.index.intersection(mkt_ret.index)
    active = (stock_ret.loc[idx] - mkt_ret.loc[idx]).astype(float)
    mu = active.rolling(window).mean()
    sd = active.rolling(window).std(ddof=1)
    return mu / sd


def _trend_ma_breakdown(monthly_px: pd.Series, ma_window: int = 6) -> bool:
    ma = monthly_px.rolling(ma_window).mean()
    if len(ma.dropna()) == 0:
        return False
    return bool(monthly_px.iloc[-1] < ma.iloc[-1])


def _pick_asof_month_end(monthly_index: pd.DatetimeIndex, as_of: str | None) -> pd.Timestamp:
    """
    as_of boleh tanggal apa saja. Karena data monthly pakai month-end,
    kita pilih month-end terakhir yang <= as_of.
    """
    if as_of is None:
        return monthly_index.max()

    as_of_ts = pd.Timestamp(as_of)
    eligible = monthly_index[monthly_index <= as_of_ts]
    if len(eligible) == 0:
        raise ValueError(f"as_of={as_of} lebih awal dari data bulanan yang tersedia.")
    return eligible.max()


def _download_daily_prices(all_tickers: list[str], start: str, end: str | None):
    data = yf.download(
        tickers=all_tickers,
        start=start,
        end=end,
        auto_adjust=False,
        progress=False,
        group_by="column",
        threads=True,
    )

    # Prefer Adj Close, fallback Close
    if isinstance(data.columns, pd.MultiIndex) and "Adj Close" in data.columns.get_level_values(0):
        px_daily = data["Adj Close"].copy()
    elif isinstance(data.columns, pd.MultiIndex) and "Close" in data.columns.get_level_values(0):
        px_daily = data["Close"].copy()
    else:
        px_daily = data.copy()

    px_daily = px_daily.dropna(how="all")
    px_daily = px_daily.loc[:, ~px_daily.columns.duplicated()]
    return px_daily


# ============================================================
# Single Live Exit Check (2-of-3)
# ============================================================
def exit_control_monthly_single_live(
    tickers: list[str],
    market_ticker: str = "^GSPC",
    start: str = "2018-01-01",
    end: str | None = None,
    as_of: str | None = None,          # <-- live check date
    rf_monthly: float = 0.0,
    beta_alpha_window: int = 12,
    risk_window: int = 12,
    alpha_3m_drop_threshold: float = -0.002,
    alpha_rank_drop_points: float = 0.20,
    sharpe_drop_threshold: float = -0.30,
    ir_drop_threshold: float = -0.30,
    vol_increase_threshold: float = 0.20,
    mom_lookback: int = 6,
    ma_window: int = 6,
    action_reduce: bool = True,
) -> tuple[pd.DataFrame, pd.Timestamp, pd.DataFrame, pd.DataFrame]:
    """
    Returns:
      - out_signals: signal table (as-of)
      - last_date: month-end date used (<= as_of)
      - px_monthly: monthly prices (all tickers + market)
      - rets_m: monthly returns
    """
    tickers = list(dict.fromkeys([t for t in tickers if isinstance(t, str) and t.strip()]))
    all_tickers = [market_ticker] + [t for t in tickers if t != market_ticker]

    px_daily = _download_daily_prices(all_tickers, start=start, end=end)
    px_monthly = _to_monthly_adjclose(px_daily)
    rets_m = _monthly_returns(px_monthly)

    if market_ticker not in rets_m.columns:
        raise ValueError(f"Market ticker {market_ticker} tidak ditemukan di data download.")

    # Pick month-end used for evaluation
    last_date = _pick_asof_month_end(rets_m.index.dropna(), as_of)

    # Slice all monthly series to last_date to avoid look-ahead
    px_monthly = px_monthly.loc[px_monthly.index <= last_date]
    rets_m = rets_m.loc[rets_m.index <= last_date]

    mkt_ret = rets_m[market_ticker].dropna()

    # Precompute rolling alpha per ticker (until last_date)
    alpha_map = {}
    sharpe_map = {}
    ir_map = {}
    vol_map = {}
    alpha_last = {}

    for t in tqdm(tickers, desc="Compute rolling metrics (single live)"):
        if t not in rets_m.columns:
            continue

        r = rets_m[t].dropna()
        idx = r.index.intersection(mkt_ret.index)
        r = r.loc[idx]
        mr = mkt_ret.loc[idx]

        _, a = _rolling_beta_alpha(r, mr, window=beta_alpha_window)
        a = a.loc[a.index <= last_date]
        alpha_map[t] = a

        sh = _rolling_sharpe(r, rf_monthly=rf_monthly, window=risk_window).loc[lambda s: s.index <= last_date]
        sharpe_map[t] = sh

        ir = _rolling_information_ratio(r, mr, window=risk_window).loc[lambda s: s.index <= last_date]
        ir_map[t] = ir

        vol = r.rolling(risk_window).std(ddof=1).loc[lambda s: s.index <= last_date]
        vol_map[t] = vol

        alpha_last[t] = _safe_float(a.dropna().iloc[-1]) if not a.dropna().empty else np.nan

    # Rank alpha cross-section at last_date (percentile)
    alpha_series_last = pd.Series(alpha_last, dtype=float).dropna()
    rank_last = alpha_series_last.rank(pct=True, ascending=True) if not alpha_series_last.empty else pd.Series(dtype=float)

    # previous month-end for rank drop
    prev_date = None
    eligible_prev = rets_m.index[rets_m.index < last_date]
    if len(eligible_prev) > 0:
        prev_date = eligible_prev.max()

    alpha_prev = {}
    for t in tickers:
        a = alpha_map.get(t)
        if a is None or prev_date is None:
            alpha_prev[t] = np.nan
            continue
        a_prev = a.loc[a.index <= prev_date]
        alpha_prev[t] = _safe_float(a_prev.dropna().iloc[-1]) if not a_prev.dropna().empty else np.nan

    alpha_series_prev = pd.Series(alpha_prev, dtype=float).dropna()
    rank_prev = alpha_series_prev.rank(pct=True, ascending=True) if not alpha_series_prev.empty else pd.Series(dtype=float)

    # Build output
    rows = []
    for t in tickers:
        if t not in rets_m.columns:
            continue

        name = _get_company_name(t)

        r = rets_m[t].dropna()
        idx = r.index.intersection(mkt_ret.index)
        r = r.loc[idx]
        mr = mkt_ret.loc[idx]

        px = px_monthly[t].dropna()

        a = alpha_map.get(t, pd.Series(dtype=float)).dropna()
        sh = sharpe_map.get(t, pd.Series(dtype=float)).dropna()
        ir = ir_map.get(t, pd.Series(dtype=float)).dropna()
        vol = vol_map.get(t, pd.Series(dtype=float)).dropna()

        # (1) Alpha melemah
        alpha_weak = False
        alpha_3m_chg = np.nan
        if len(a) >= 4:
            alpha_3m_chg = _safe_float(a.iloc[-1] - a.iloc[-4])
            if np.isfinite(alpha_3m_chg) and alpha_3m_chg <= alpha_3m_drop_threshold:
                alpha_weak = True

        rank_now = _safe_float(rank_last.get(t, np.nan)) if not rank_last.empty else np.nan
        rank_prev_t = _safe_float(rank_prev.get(t, np.nan)) if not rank_prev.empty else np.nan
        rank_drop = np.nan
        rank_drop_flag = False
        if np.isfinite(rank_now) and np.isfinite(rank_prev_t):
            rank_drop = rank_now - rank_prev_t
            if rank_drop <= -alpha_rank_drop_points:
                rank_drop_flag = True

        cond1 = bool(alpha_weak or rank_drop_flag)

        # (2) Risk-adjusted return turun
        sharpe_delta = np.nan
        ir_delta = np.nan
        vol_ratio = np.nan
        ra_weak = False

        if len(sh) >= 2:
            sharpe_delta = _safe_float(sh.iloc[-1] - sh.iloc[-2])
        if len(ir) >= 2:
            ir_delta = _safe_float(ir.iloc[-1] - ir.iloc[-2])

        if len(vol) >= 2 and np.isfinite(vol.iloc[-2]) and vol.iloc[-2] != 0:
            vol_ratio = _safe_float((vol.iloc[-1] / vol.iloc[-2]) - 1.0)

        last_ret = _safe_float(r.iloc[-1]) if len(r) >= 1 else np.nan

        sharp_or_ir_drop = (
            (np.isfinite(sharpe_delta) and sharpe_delta <= sharpe_drop_threshold) or
            (np.isfinite(ir_delta) and ir_delta <= ir_drop_threshold)
        )
        vol_spike = (np.isfinite(vol_ratio) and vol_ratio >= vol_increase_threshold)
        no_comp = (np.isfinite(last_ret) and last_ret <= 0.0)

        if sharp_or_ir_drop or (vol_spike and no_comp):
            ra_weak = True

        cond2 = bool(ra_weak)

        # (3) Momentum melemah
        mom_weak = False
        rel_mom = np.nan

        px_mkt = px_monthly[market_ticker].dropna()
        if len(px) >= mom_lookback + 1 and len(px_mkt) >= mom_lookback + 1:
            common_idx = px.index.intersection(px_mkt.index)
            px2 = px.loc[common_idx]
            pxm2 = px_mkt.loc[common_idx]
            if len(px2) >= mom_lookback + 1:
                stock_mom = _safe_float(px2.iloc[-1] / px2.iloc[-(mom_lookback + 1)] - 1.0)
                mkt_mom = _safe_float(pxm2.iloc[-1] / pxm2.iloc[-(mom_lookback + 1)] - 1.0)
                rel_mom = stock_mom - mkt_mom
                if np.isfinite(rel_mom) and rel_mom < 0:
                    mom_weak = True

        ma_break = False
        if len(px) >= ma_window:
            ma_break = _trend_ma_breakdown(px, ma_window=ma_window)

        cond3 = bool(mom_weak or ma_break)

        # Rule 2-of-3
        cond_count = int(cond1) + int(cond2) + int(cond3)
        if cond_count >= 2:
            action = "SELL/REDUCE" if action_reduce else "SELL"
        else:
            action = "HOLD"

        rows.append({
            "AsOf_MonthEnd": last_date.date().isoformat(),
            "Ticker": t,
            "Company Name": name,
            "Cond1_AlphaWeak": cond1,
            "Cond2_RiskAdjWeak": cond2,
            "Cond3_MomentumWeak": cond3,
            "Conditions_Met": cond_count,
            "Action": action,
            "Alpha_3M_Change": alpha_3m_chg,
            "AlphaRank_Now": rank_now,
            "AlphaRank_Prev": rank_prev_t,
            "AlphaRank_Delta": rank_drop,
            "Sharpe_Delta_1M": sharpe_delta,
            "IR_Delta_1M": ir_delta,
            "Vol_Ratio_Change": vol_ratio,
            "RelMom_6M_vs_Mkt": rel_mom,
            "LastMonthReturn": last_ret,
            "MA_Breakdown": ma_break,
        })

    out = pd.DataFrame(rows)

    # ===============================
    # FORCE OUTPUT ORDER = INPUT TICKERS
    # ===============================
    ticker_order = {t: i for i, t in enumerate(tickers)}

    if not out.empty:
        out["__order"] = out["Ticker"].map(ticker_order).astype(int)
        out = (
        out
        .sort_values("__order")
        .drop(columns="__order")
        .reset_index(drop=True)
    )

    return out, last_date, px_monthly, rets_m


def build_last_n_months_price_table(
    px_monthly: pd.DataFrame,
    tickers: list[str],
    market_ticker: str,
    asof_month_end: pd.Timestamp,
    n_months: int = 6
) -> pd.DataFrame:
    """
    Table harga monthly (month-end) untuk n bulan terakhir (plus 1 baris ekstra).
    """
    # include market + tickers in desired order
    cols = [market_ticker] + [t for t in tickers if t != market_ticker]
    cols = [c for c in cols if c in px_monthly.columns]

    px2 = px_monthly.loc[px_monthly.index <= asof_month_end, cols].copy()
    idx_sorted = px2.index.sort_values()

    k = min(len(idx_sorted), n_months + 1)   # +1 supaya terlihat continuity
    last_dates = idx_sorted[-k:]             # <-- FIX: slicing, bukan .tail()

    px2 = px2.loc[last_dates]

    # pretty index (YYYY-MM-DD)
    px2.index = px2.index.strftime("%Y-%m-%d")
    return px2


# ============================================================
# RUN
# ============================================================
if __name__ == "__main__":
    df_live, asof_month_end, px_m, rets_m = exit_control_monthly_single_live(
        tickers=CONFIG["TICKERS"],
        market_ticker=CONFIG["MARKET_TICKER"],
        start=CONFIG["DOWNLOAD_START"],
        end=CONFIG["DOWNLOAD_END"],
        as_of=CONFIG["AS_OF_DATE"],
        rf_monthly=CONFIG["RF_MONTHLY"],
        beta_alpha_window=CONFIG["BETA_ALPHA_WINDOW"],
        risk_window=CONFIG["RISK_WINDOW"],
        alpha_3m_drop_threshold=CONFIG["ALPHA_3M_DROP_THRESHOLD"],
        alpha_rank_drop_points=CONFIG["ALPHA_RANK_DROP_POINTS"],
        sharpe_drop_threshold=CONFIG["SHARPE_DROP_THRESHOLD"],
        ir_drop_threshold=CONFIG["IR_DROP_THRESHOLD"],
        vol_increase_threshold=CONFIG["VOL_INCREASE_THRESHOLD"],
        mom_lookback=CONFIG["MOM_LOOKBACK"],
        ma_window=CONFIG["MA_WINDOW"],
        action_reduce=CONFIG["ACTION_REDUCE"],
    )

    print(f"\n=== SINGLE LIVE CHECK ===")
    print(f"Live check date  : {CONFIG['AS_OF_DATE']}")
    print(f"Used month-end   : {asof_month_end.date().isoformat()} (<= live check date)")

    cols_show = [
        "Ticker", "Company Name",
        "Cond1_AlphaWeak", "Cond2_RiskAdjWeak", "Cond3_MomentumWeak",
        "Conditions_Met", "Action",
        "Alpha_3M_Change", "RelMom_6M_vs_Mkt", "LastMonthReturn"
    ]
    if df_live.empty:
        print("\nNo output (cek ticker / tanggal / data).")
    else:
        print("\n=== SIGNAL TABLE (as-of) ===")
        print(df_live[cols_show].to_string(index=False))

    # Price table (last N months)
    n = int(CONFIG["SHOW_LAST_N_MONTHS_PRICES"])
    px_table = build_last_n_months_price_table(
        px_monthly=px_m,
        tickers=CONFIG["TICKERS"],
        market_ticker=CONFIG["MARKET_TICKER"],
        asof_month_end=asof_month_end,
        n_months=n
    )
    print(f"\n=== MONTHLY PRICES (last ~{n} months up to {asof_month_end.date().isoformat()}) ===")
    print(px_table.to_string())


Compute rolling metrics (single live): 100%|██████████| 10/10 [00:00<00:00, 31.30it/s]



=== SINGLE LIVE CHECK ===
Live check date  : 2024-11-01
Used month-end   : 2024-10-31 (<= live check date)

=== SIGNAL TABLE (as-of) ===
Ticker                    Company Name  Cond1_AlphaWeak  Cond2_RiskAdjWeak  Cond3_MomentumWeak  Conditions_Met Action  Alpha_3M_Change  RelMom_6M_vs_Mkt  LastMonthReturn
  TMUS               T-Mobile US, Inc.            False              False               False               0   HOLD         0.014751          0.235963         0.081411
    SO          Southern Company (The)             True              False               False               1   HOLD         0.015749          0.127223         0.009426
   WMT                    Walmart Inc.            False              False               False               0   HOLD         0.015537          0.256518         0.014861
     T                       AT&T Inc.             True              False               False               1   HOLD         0.003157          0.238937         0.037700
   PGR   Pro